In [ ]:
# import
import requests
import json

# default variables
dbName = "j1"
url = "http://cloud.gstore.cn/api"
key = "==="
secret = "==="
headers = {
    'Content-Type': "application/json",
}
params = {
    'accesskeyid': key,
    'access_secret': secret
}


In [ ]:
# tools
def showDB():
    p = params
    params['action'] = "showDB"
    response = requests.request(method="POST", url=url, json=p, verify=False)
    result_json = response.json()
    print(result_json['list'])
def wrap(name):
    return "<file:///F:/d2r-server-0.7/holder8.nt#holder_copy/"+name+">"
def unwrap(name):
    prefix = "file:///F:/d2r-server-0.7/holder8.nt#holder_copy/"
    return name.replace(prefix, "")

In [ ]:
# 查询两个公司之间的关联路径（2-hop）
# 例如输入公司“招商局轮船股份有限公司”和“招商银行股份有限公司”，得到这两家公司之间的所有路径。
def two_hop_path(A, B):
    p = params
    p['action'] = "queryDB"
    p['dbName'] = dbName
    sql_str = "SELECT ?z ?h WHERE { "\
        + "{" + wrap(A) + "  <http://localhost:2020/vocab/resource/holder_copy_holder_name>	 ?z . ?z  <http://localhost:2020/vocab/resource/holder_copy_holder_name>	 " + wrap(B) + ".}" \
        + " UNION " \
        + "{" + wrap(A) + " ?h " + wrap(B) + ".}" \
        + " }"
    p['sparql'] = sql_str
    response = requests.post(url=url, json=p, verify=False)
    result_json = response.json()
    res = result_json['data']['results']['bindings']
    for i in res:
        if 'z' in i:
            print(A+" -> "+unwrap(i['z']['value'])+" -> "+B)
        if 'h' in i:
            print(A+" -> "+B)
def task1_two_hop_path(A, B):
    two_hop_path(A, B)
    two_hop_path(B, A)


In [ ]:
# 多层股权的穿透式查询，可以根据指定层数获得对应层级的股东
# 例如：输入“招商局轮船股份有限公司””和层数3，就会把“招商局轮船股份有限公司”所对应公司所有三层以内的公司找出来。
def n_layer_who_hold_me(N, name):
    p = params
    p['action'] = "queryDB"
    p['dbName'] = dbName
    sql_str_start = "SELECT * WHERE { "
    sql_str_end = " }"
    sql_str_body = ""
    if (N == 1):
        sql_str_body = "{?a  <http://localhost:2020/vocab/resource/holder_copy_holder_name>	 " + wrap(name) + ".}"
    else:
        sql_str_body = "{?a  <http://localhost:2020/vocab/resource/holder_copy_holder_name>	 ?b1. "
        for i in range(2, N):
            sql_str_body += " ?b" + str(i-1) + "  <http://localhost:2020/vocab/resource/holder_copy_holder_name>	 ?b" + str(i) + ". "
        sql_str_body += " ?b" + str(N-1) + "  <http://localhost:2020/vocab/resource/holder_copy_holder_name>	 " + wrap(name) + ".}"
    sql_str = sql_str_start + sql_str_body + sql_str_end
    p['sparql'] = sql_str
    response = requests.post(url=url, json=p, verify=False)
    result_json = response.json()
    res = result_json['data']['results']['bindings']
    s = set()
    for i in res:
        if 'a' in i:
            s.add(unwrap(i['a']['value']))
    for i in s:
        print(i)

def task_2_n_layer_who_hold_me(N, name):
    for i in range(1, N+1):
        print("第" + str(i) + "层股东：")
        n_layer_who_hold_me(i, name)
        print("\n")

In [ ]:
# 环形持股查询，判断两家公司是否存在环形持股现象，环形持股是指两家公司彼此持有对方的股份。
# 例如：输入“A”和“C”，判断两家公司是否存在环形持股。
def n_hop(N, A, B):
    p = params
    p['action'] = "queryDB"
    p['dbName'] = dbName
    sql_str_start = "SELECT * WHERE { "
    sql_str_end = " }"
    sql_str_body = ""
    if (N == 1):
        sql_str_body = "{"+wrap(A)+"  <http://localhost:2020/vocab/resource/holder_copy_holder_name>	 " + wrap(B) + ".}"
    else:
        sql_str_body = "{"+wrap(A)+" <http://localhost:2020/vocab/resource/holder_copy_holder_name>	 ?b1. "
        for i in range(2, N):
            sql_str_body += " ?b" + str(i-1) + "  <http://localhost:2020/vocab/resource/holder_copy_holder_name>  ?b" + str(i) + ". "
        sql_str_body += " ?b" + str(N-1) + "  <http://localhost:2020/vocab/resource/holder_copy_holder_name>	 " + wrap(B) + ".}"
    sql_str = sql_str_start + sql_str_body + sql_str_end
    p['sparql'] = sql_str
    response = requests.post(url=url, json=p, verify=False)
    result_json = response.json()
    res = result_json['data']['results']['bindings']
    if len(res) > 0:
        return True
    else:
        return False

def task3_ring(A, B):
    flag_a2b = False
    flag_b2a = False
    for i in range(1, 6):
        if n_hop(i, A, B):
            flag_a2b = True
            break
    for i in range(1, 6):
        if n_hop(i, B, A):
            flag_b2a = True
            break
        
    if flag_a2b and flag_b2a:
        print("存在环形持股")
    else:
        print("不存在环形持股")


In [ ]:
# test connection
showDB()

In [ ]:
# task1
task1_two_hop_path("招商局轮船股份有限公司", "招商银行股份有限公司")

In [ ]:
# task2
task_2_n_layer_who_hold_me(3, "招商局轮船股份有限公司")
task_2_n_layer_who_hold_me(2, "安邦人寿保险股份有限公司")


In [ ]:
# task3
task3_ring("A", "C")
task3_ring("A", "E")